Trivia QA

In [5]:
from sentence_transformers import SentenceTransformer, util
import os
import json
from predictionguard import PredictionGuard
from getpass import getpass
import pandas as pd
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [3]:
load_dotenv()
pg_access_token = os.getenv('PG_API_KEY')
os.environ['PREDICTIONGUARD_API_KEY'] = pg_access_token
client = PredictionGuard()

In [1]:
#Run this cell if the data is not already in SQuAD.csv
# splits = {'train': 'plain_text/train-00000-of-00001.parquet', 'validation': 'plain_text/validation-00000-of-00001.parquet'}
# df = pd.read_parquet("hf://datasets/rajpurkar/squad/" + splits["train"])
# df.to_csv('SQuAD.csv')

In [4]:
df = pd.read_csv('SQuAD_space_race.csv')
ls_questions = list(df['question'])
ls_answers = list(df['answers'])

In [6]:
# Load the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [43]:
def get_response_from_model(inputs):
    # print(inputs)
    prompt = """### Instruction:
    Answer the question in one word

    ### question:
    {inputs}

    ### Response:
    """.format(inputs=inputs)

    response = client.completions.create(
        model="Hermes-2-Pro-Llama-3-8B",
        prompt=prompt
    )

    filtered_response = response['choices'][0]['text'].split('\n')[0].strip()

    filtered_response = filtered_response.split('#')[0]
    return filtered_response
def create_prompt_template_variations(question):
    # Define your prompt templates
    prompt_templates = [
        # Basic template
        """### Instruction:
        Answer the question in one word
        ### Question:
        {question}
        ### Response:""",
        
        # More casual
        """Hey! Can you answer this with just one word?
        Question: {question}
        Answer:""",
        
        # More formal
        """You are tasked with providing a single-word response to the following query:
        {question}
        Response:""",
        
        # Minimal
        """{question}
        Answer in one word:""",
        
        # With emphasis
        """### Important: Use ONLY ONE WORD to answer
        Question: {question}
        Answer:""",
    ]
    return prompt_templates
def test_prompt_sensitivity(question, client, prompts):
    """Test model's responses to different prompt formulations."""
    results = []
    
    for prompt in prompts[0]:
        response = client.completions.create(
            model="Hermes-2-Pro-Llama-3-8B",
            prompt=prompt
        )
        results.append(response['choices'][0]['text'].strip())
    
    return results
def calculate_semantic_similarity(responses):
    """
    Calculate the average pairwise semantic similarity between responses
    """
    if not responses:
        return 0.0
    
    # Get embeddings for all responses
    embeddings = model.encode(responses)
    
    # Calculate pairwise similarities
    similarities = cosine_similarity(embeddings)
    
    # Get average similarity (excluding self-similarity)
    n = len(responses)
    if n <= 1:
        return 1.0
    
    # Calculate mean of upper triangle (excluding diagonal)
    total_similarity = 0
    count = 0
    for i in range(n):
        for j in range(i + 1, n):
            total_similarity += similarities[i][j]
            count += 1
    
    return total_similarity / count if count > 0 else 0.0
def create_random_variations(question):
    import random
    import string
    
    variations = []
    for i in range(3):
        # Convert question to list of characters for easier manipulation
        chars = list(question)
        
        # Randomly select 2 positions for letter replacement
        letter_positions = random.sample([i for i, c in enumerate(chars) if c.isalpha()], 2)
        
        # Replace letters with random letters
        for pos in letter_positions:
            chars[pos] = random.choice(string.ascii_lowercase)
        
        # Randomly select 1 position for capitalization
        cap_position = random.choice([i for i, c in enumerate(chars) if c.isalpha()])
        chars[cap_position] = chars[cap_position].upper()
        
        # Join back into string and add to variations
        variations.append(''.join(chars))
    
    return variations

In [49]:
def test_random_variations():
    full_responses = []
    for question in ls_questions[:2]: 
        responses = []
        main_response = get_response_from_model(question)
        responses.append(main_response)
        print('Question: ', question)
        print('Main Response: ', main_response)
        question_variations = create_random_variations(question)
        for variation in question_variations:
            
            print('Variation: ', variation)
            response = get_response_from_model(variation)
            print('Response: ', response)
            responses.append(response)
        full_responses.append(responses)
        # print('Semantic Similarity: ', calculate_semantic_similarity(responses))

    return full_responses


In [50]:
full_responses = test_random_variations()
full_responses

Question:  Whose technology enabled the Space Race between the Soviet Union and the united States?
Main Response:  German
Variation:  Whose technology enabled the Space Race bftween the Soviet Union and the tnited States?
Response:  Germany
Variation:  Whose technqlogy enabled the Space Race between the Soviet Uncon and the united StAtes?
Response:  Soviet
Variation:  Whose techNology enabved the rpace Race between the Soviet Union and the united States?
Response:  Soviet Unions or USA both used it?
Question:  Who was able to launch the first orbiting satellite?
Main Response:  Russia
Variation:  Who tas able to launch the fiRst orbiting sadellite?
Response:  Soviet Union
Variation:  Who was able bo lAunch ehe first orbiting satellite?
Response:  The Soviet Union
Variation:  Who was aBle to launch the lirst orbiting satellhte?
Response:  Sergey Korolev 


[['German', 'Germany', 'Soviet', 'Soviet Unions or USA both used it?'],
 ['Russia', 'Soviet Union', 'The Soviet Union', 'Sergey Korolev ']]